In [4]:
import torch
import torch.nn as nn
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url
    
from typing import Any

# AlexNet 모델 : 8개의 계층 구조, ReLU 함수를 사용한 첫 모델

__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


class AlexNet(nn.Module):

    def __init__(self, num_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            #Conv1
            #input channel : 3 , output channel : 64, kernel_size : 11*11, stride : 4, padding : 2
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True), # inplace=True 하면, inplace 연산을 수행함, inplace 연산은 결과값을 새로운 변수에 값을 저장하는 대신 기존의 데이터를 대체하는것을 의미
            #Max Pool1
            nn.MaxPool2d(kernel_size=3, stride=2),
            #Conv2
            #input channel : 64 , output channel : 192, kernel_size : 5*5,  padding : 2
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            #Max Pool2
            nn.MaxPool2d(kernel_size=3, stride=2),
            #Conv3
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            ##Conv4
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #Conv5
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #Max Pool3
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential( # sequential로 레이들을 쌓음)
            #드롭아웃 
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096), # 이전 fully connected layer의 output dimension = 4096
            nn.ReLU(inplace=True), 
            nn.Linear(4096, num_classes), # 최종 분류 층 / fully connected layer의 output dimension 과 클래스 개수를 맞춰줌
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #특징 추출 부분
        x = self.features(x)
       
        x = self.avgpool(x)
        #output shape : (batch size * 256(channel), 6, 6)
        #Flatten
        x = torch.flatten(x, 1)
        #output shape (batch_size, 256 * 6* 6)
        #분류 분류 
        x = self.classifier(x)
        return x


def alexnet(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> AlexNet:
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

In [5]:
import tensorflow as tf

def AlexNet(
  input_shape=None,
  weights=None,
  classes=1000,
  classifier_activation='softmax'):
  
  model = tf.keras.Sequential([
      #특징 추출 부분 
      #Conv 1
      tf.keras.layers.Conv2D(filters=96,
                              kernel_size=(11, 11),
                              strides=4,
                              padding="valid",
                              activation=tf.keras.activations.relu,
                              input_shape=input_shape),
      #Max Pool 1
      tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                strides=2,
                                padding="valid"),
      tf.keras.layers.BatchNormalization(),
      #Conv 2
      tf.keras.layers.Conv2D(filters=256,
                              kernel_size=(5, 5),
                              strides=1,
                              padding="same", # 제로 패딩을 수행하겠다. / 입력으로 들어온 크기와 아웃풋
                              activation=tf.keras.activations.relu),
      #Max Pool 2
      tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                strides=2,
                                padding="same"),
      tf.keras.layers.BatchNormalization(),
      #Conv 3
      tf.keras.layers.Conv2D(filters=384,
                              kernel_size=(3, 3),
                              strides=1,
                              padding="same",
                              activation=tf.keras.activations.relu),
      #Conv 4
      tf.keras.layers.Conv2D(filters=384,
                              kernel_size=(3, 3),
                              strides=1,
                              padding="same",
                              activation=tf.keras.activations.relu),
      #Conv 5
      tf.keras.layers.Conv2D(filters=256,
                              kernel_size=(3, 3),
                              strides=1,
                              padding="same",
                              activation=tf.keras.activations.relu),
      #Max Pool 3
      tf.keras.layers.MaxPool2D(pool_size=(3, 3), 
                                strides=2,
                                padding="same"), # 특징 맵 뽑기
      tf.keras.layers.BatchNormalization(),
      
      tf.keras.layers.Flatten(), # flatten 연산 수행. flatten 은 넘파이에서 제공하는, 다차원 배열 공간을 1차원으로 평탄화 해주는 함수이다. 
      
      #분류 층 부분
      #Fully connected layer 1 
      tf.keras.layers.Dense(units=4096,
                            activation=tf.keras.activations.relu), #output dimension이 4096
      tf.keras.layers.Dropout(rate=0.2),
      #Fully connected layer 2
      tf.keras.layers.Dense(units=4096,
                            activation=tf.keras.activations.relu),
      tf.keras.layers.Dropout(rate=0.2),
      
      #Fully connected layer 3
      tf.keras.layers.Dense(units=classes, # 학습에 사용한 데이터셋의 클래스 개수에 맞게 output dimension을 지정
                            activation=tf.keras.activations.softmax) #softmax 확률값을 출력해야 하므로 이렇게 적음. 
  ])

  return model